<a href="https://colab.research.google.com/github/udupa-varun/pyimagesearch_uni/blob/main/deep_learning/101/first_image_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com/first-image-classifier/first-image-classifier.zip
!unzip -qq first-image-classifier.zip
%cd first-image-classifier

--2022-06-07 12:45:52--  https://pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com/first-image-classifier/first-image-classifier.zip
Resolving pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com (pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com)... 52.218.243.113
Connecting to pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com (pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com)|52.218.243.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 196971354 (188M) [application/zip]
Saving to: ‘first-image-classifier.zip’

first-image-classif 100%[===================>] 187.85M  21.2MB/s    in 9.6s    

2022-06-07 12:46:02 (19.5 MB/s) - ‘first-image-classifier.zip’ saved [196971354/196971354]

/content/first-image-classifier


In [2]:
# import the necessary packages
import os

import argparse
import cv2
import numpy as np
from imutils import paths
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [3]:
class SimplePreprocessor:
    def __init__(self, width, height, inter=cv2.INTER_AREA):
        self.width = width
        self.height = height
        self.inter = inter

    def preprocess(self, image):
        # resize image to a fixed size, ignoring aspect ratio
        return cv2.resize(image, (self.width, self.height), interpolation=self.inter)

In [4]:
class SimpleDatasetLoader:
    def __init__(self, preprocessors=None):
        # store the image preprocessor
        self.preprocessors = preprocessors

        # if the preprocessors are None, initialize them as an
        # empty list
        if self.preprocessors is None:
            self.preprocessors = []

    def load(self, image_paths, verbose=-1):
        # initialize the list of features and labels
        data = []
        labels = []

        # loop over the input images
        for (i, image_path) in enumerate(image_paths):
            # load the image and extract the class label assuming
            # that our path has the following format:
            # /path/to/dataset/{class}/{image}.jpg
            image = cv2.imread(image_path)
            label = image_path.split(os.path.sep)[-2]

            # check to see if our preprocessors are not None
            if self.preprocessors is not None:
                # loop over the preprocessors and apply each to
                # the image
                for p in self.preprocessors:
                    image = p.preprocess(image)

            # treat our processed image as a "feature vector"
            # by updating the data list followed by the labels
            data.append(image)
            labels.append(label)

            # show an update every `verbose` images
            if verbose > 0 and i > 0 and (i + 1) % verbose == 0:
                print(f"[INFO] processed {i + 1}/{len(image_paths)}")

        # return a tuple of the data and labels
        return (np.array(data), np.array(labels))

In [5]:
# construct the argument parse and parse the arguments
#ap = argparse.ArgumentParser()
#ap.add_argument("-d", "--dataset", required=True,
#	help="path to input dataset")
#ap.add_argument("-k", "--neighbors", type=int, default=1,
#	help="# of nearest neighbors for classification")
#ap.add_argument("-j", "--jobs", type=int, default=-1,
#	help="# of jobs for k-NN distance (-1 uses all available cores)")
#args = vars(ap.parse_args())

# since we are using Jupyter Notebooks we can replace our argument
# parsing code with *hard coded* arguments and values
args = {
    "dataset": "dataset/animals",
    "neighbors": 1,
    "jobs": -1
}

In [7]:
# grab list of images to describe
print("[INFO] loading images...")
image_paths = list(paths.list_images(args["dataset"]))

# init image preprocessor
sp = SimplePreprocessor(32, 32)
sdl = SimpleDatasetLoader(preprocessors=[sp])
# load dataset and reshape data matrix
(data, labels) = sdl.load(image_paths, verbose=500)
data = data.reshape((data.shape[0], 3072))

# show memory consumption of the images
mem_usage = data.nbytes / (1024 * 1024.0)
print(f"[INFO] features matrix: {mem_usage:.1f}MB")

[INFO] loading images...
[INFO] processed 500/3000
[INFO] processed 1000/3000
[INFO] processed 1500/3000
[INFO] processed 2000/3000
[INFO] processed 2500/3000
[INFO] processed 3000/3000
[INFO] features matrix: 8.8MB


In [8]:
# encode labels as integers
le = LabelEncoder()
labels = le.fit_transform(labels)

# partition the data into train/test splits (75/25)
(train_x, test_x, train_y, test_y) = train_test_split(data, labels, test_size=0.25, random_state=42)

In [9]:
# train and evaluate a k-NN classifier on raw pixel intensities
print("[INFO] evaluating a k-NN classifier...")
model = KNeighborsClassifier(n_neighbors=args["neighbors"], n_jobs=args["jobs"])
model.fit(train_x, train_y)
print(classification_report(test_y, model.predict(test_x), target_names=le.classes_))

[INFO] evaluating a k-NN classifier...
              precision    recall  f1-score   support

        cats       0.39      0.49      0.44       239
        dogs       0.37      0.46      0.41       249
       panda       0.78      0.41      0.54       262

    accuracy                           0.45       750
   macro avg       0.51      0.45      0.46       750
weighted avg       0.52      0.45      0.46       750



For a detailed walkthrough of the concepts and code, be sure to refer to the full tutorial, [*Your First Image Classifier: Using k-NN to Classify Images*](https://www.pyimagesearch.com/2021/04/17/your-first-image-classifier-using-k-nn-to-classify-images/) published on 2021-04-17.